In [44]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [3]:

class MDB(nn.Module): 
    def __init__(self,input,hiden):  
        super(MDB,self).__init__()
        self.conv1=nn.Sequential(  
            nn.Conv2d(in_channels = input,out_channels = hiden,kernel_size = 3 , stride = 1,padding=1),  
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),  
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 3 , stride = 1,padding=2,dilation=2),  
            nn.BatchNorm2d(hiden),   
            nn.ReLU(),   
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 3 , stride = 1,padding=2,dilation=2),  
            nn.BatchNorm2d(hiden),  
	        nn.ReLU()  
      )  
        self.conv2=nn.Sequential(  
            nn.Conv2d(in_channels = input,out_channels = hiden,kernel_size = 5 , stride = 1,padding=2),  
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),  
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 5 , stride = 1,padding=4,dilation=2),  
            nn.BatchNorm2d(hiden),   
            nn.ReLU(),   
            nn.Conv2d(in_channels = hiden,out_channels =hiden,kernel_size = 5 , stride = 1,padding=4,dilation=2),  
            nn.BatchNorm2d(hiden),  
	        nn.ReLU()  
      )  
        self.conv3=nn.Sequential(
            nn.Conv2d(in_channels=input,out_channels=hiden,kernel_size=1,stride=1,padding=0),
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),
        )
    def forward(self ,x):
        p1=self.conv1(x)
        p2=self.conv2(x)
        p3=self.conv3(x)
        x=F.relu(torch.concat((p1,p2,p3),dim=1))
        return x 



In [76]:

class ChannelAttentionModule(nn.Module):
    def __init__(self, channel, ratio=16):
        super(ChannelAttentionModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.shared_MLP = nn.Sequential(
            nn.Conv2d(channel, channel // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channel // ratio, channel, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avgout = self.shared_MLP(self.avg_pool(x))
        maxout = self.shared_MLP(self.max_pool(x))
        return self.sigmoid(avgout + maxout)


class SpatialAttentionModule(nn.Module):
    def __init__(self):
        super(SpatialAttentionModule, self).__init__()
        self.conv2d = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avgout = torch.mean(x, dim=1, keepdim=True)
        maxout, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avgout, maxout], dim=1)
        out = self.sigmoid(self.conv2d(out))
        return out


class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttentionModule(channel)
        self.spatial_attention = SpatialAttentionModule()

    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out



In [77]:
#多尺度加注意力
class MODEL(nn.Module):
    def __init__(self,input,hiden,output):  
        super(MODEL,self).__init__()
        self.layer1=MDB(input,hiden)
        self.layer2=MDB(input,hiden)
        self.layer3=MDB(input,hiden)
        self.attention1=CBAM(3*hiden)
        self.attention2=CBAM(3*hiden)
        self.attention3=CBAM(3*hiden)
        self.output=nn.Linear(9*hiden,output)
        self.softmax=nn.Softmax(dim=1)
    def forward(self,x):
        p1=self.layer1(x)
        p2=self.layer2(x)
        p3=self.layer3(x)
        p1_a=self.attention1(p1)
        p2_a=self.attention2(p2)
        p3_a=self.attention3(p3)
        x=F.relu(torch.concat((p1_a,p2_a,p3_a),dim=1))
        x=F.adaptive_avg_pool2d(x,(1,1))
        x=x.view(len(x),x.shape[1])
        x=self.output(x)
        x=self.softmax(x)
        return x

In [150]:
import torch.nn as nn

#cnn
class Simple_CNN(nn.Module):
    def __init__(self,inputs, class_num):
        super(Simple_CNN, self).__init__()
        self.class_num = class_num
        self.conv1 = nn.Sequential(
            nn.Conv2d(		# input: 3,400,600
                in_channels=inputs,
                out_channels=8,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.Conv2d(
                in_channels=8,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.AvgPool2d(2),  # 16,400,600 --> 16,200,300
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.Conv2d(
                in_channels=16,
                out_channels=8,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.AvgPool2d(2),  # 8,200,300 --> 8,100,150
            nn.BatchNorm2d(8),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=8,
                out_channels=8,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.Conv2d(
                in_channels=8,
                out_channels=1,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.AvgPool2d(2),  
            nn.BatchNorm2d(1),
            nn.LeakyReLU()
        )
        self.line = nn.Sequential(
            nn.Linear(
                in_features=16,
                out_features=self.class_num
            ),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv1(x)

        x = x.view(-1, 16)
        y = self.line(x)
        return y


In [153]:
#多尺度
class MDB_class(nn.Module): 
    def __init__(self,input,hiden,classnum):  
        super(MDB_class,self).__init__()
        self.conv1=nn.Sequential(  
            nn.Conv2d(in_channels = input,out_channels = hiden,kernel_size = 3 , stride = 1,padding=1),  
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),  
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 3 , stride = 1,padding=2,dilation=2),  
            nn.BatchNorm2d(hiden),   
            nn.ReLU(),   
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 3 , stride = 1,padding=2,dilation=2),  
            nn.BatchNorm2d(hiden),  
	        nn.ReLU()  
      )  
        self.conv2=nn.Sequential(  
            nn.Conv2d(in_channels = input,out_channels = hiden,kernel_size = 5 , stride = 1,padding=2),  
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),  
            nn.Conv2d(in_channels = hiden,out_channels = hiden,kernel_size = 5 , stride = 1,padding=4,dilation=2),  
            nn.BatchNorm2d(hiden),   
            nn.ReLU(),   
            nn.Conv2d(in_channels = hiden,out_channels =hiden,kernel_size = 5 , stride = 1,padding=4,dilation=2),  
            nn.BatchNorm2d(hiden),  
	        nn.ReLU()  
      )  
        self.conv3=nn.Sequential(
            nn.Conv2d(in_channels=input,out_channels=hiden,kernel_size=1,stride=1,padding=0),
            nn.BatchNorm2d(hiden),  
            nn.ReLU(),
        )
        self.line = nn.Sequential(
            nn.Linear(
                in_features=3*hiden*32*32,
                out_features=classnum
            ),
            nn.Softmax(dim=1)
        )

    def forward(self ,x,hiden):
        p1=self.conv1(x)
        p2=self.conv2(x)
        p3=self.conv3(x)
        x=F.relu(torch.concat((p1,p2,p3),dim=1))
        x=x.view(-1,3*hiden*32*32)
        x=self.line(x)
        return x 

In [133]:

data_f10=pd.read_csv('实作-1107\数据\F10.csv',header=None).dropna()
data_f5=pd.read_csv('实作-1107\数据\F5.csv',header=None).dropna()
data_f15=pd.read_csv('实作-1107\数据\F15.csv',header=None).dropna()
data_normal=pd.read_csv('实作-1107\数据\F15.csv',header=None).dropna()


In [ ]:
data_normal_train=[]
for i in range(30):
    fre = np.array(data_f5.iloc[:,1])[i*1024:(i+1)*1024]
    result = fre.reshape((32, 32))
    result =255*(result - np.min(result)) / (np.max(result) - np.min(result))
    data_normal_train.append(result)
data_normal_train=np.array(data_normal_train).astype(int)
for i  in  range (len(data_normal_train)):
    plt.imshow(data_normal_train[i],cmap="gray")
    plt.show()


In [ ]:
data_f5_train=[]
for i in range(30):
    fre = np.array(data_f5.iloc[:,1])[i*1024:(i+1)*1024]
    result = fre.reshape((32, 32))
    result =255*(result - np.min(result)) / (np.max(result) - np.min(result))
    data_f5_train.append(result)
data_f5_train=np.array(data_f5_train).astype(int)
for i  in  range (len(data_f5_train)):
    plt.imshow(data_f5_train[i],cmap="gray")
    plt.show()


In [ ]:
data_f10_train=[]
for i in range(30):
    fre = np.array(data_f10.iloc[:,1])[i*1024:(i+1)*1024]
    result = fre.reshape((32, 32))
    result = 255*(result - np.min(result)) / (np.max(result) - np.min(result))
    data_f10_train.append(result)
data_f10_train=np.array(data_f10_train).astype(int)
for i  in  range (len(data_f10_train)):
    plt.imshow(data_f10_train[i],cmap="gray")
    plt.show()


In [ ]:
data_f15_train=[]
for i in range(30):
    fre = np.array(data_f15.iloc[:,1])[i*1024:(i+1)*1024]
    result = fre.reshape((32, 32))
    result = 255*(result - np.min(result)) / (np.max(result) - np.min(result))
    data_f15_train.append(result)
data_f15_train=np.array(data_f15_train).astype(int)
for i  in  range (len(data_f15_train)):
    plt.imshow(data_f15_train[i],cmap="gray")
    plt.show()


In [59]:
data_f15_train=torch.from_numpy(data_f15_train).float()
data_f10_train=torch.from_numpy(data_f10_train).float()
data_f5_train=torch.from_numpy(data_f5_train).float()
data_normal_train=torch.from_numpy(data_normal_train).float()

In [60]:
data_train=torch.concat((data_f5_train,data_f10_train,data_f15_train,data_normal_train),dim=0)
data_train=data_train.reshape(120,1,32,32)

In [69]:

from torch.utils.data import TensorDataset
labels=torch.concat((torch.tensor([0]*30),torch.tensor([1]*30),torch.tensor([2]*30),torch.tensor([3]*30)))
data = TensorDataset(data_train, labels)

In [71]:
from torch.utils.data import SubsetRandomSampler
batch_size = 12
validation_split = 0.2
shuffle_dataset = True
random_seed= 42

dataset_size = len(data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(data, batch_size=batch_size,
                                                sampler=valid_sampler)

In [99]:
#卷积层加多尺度
model=MODEL(1,64,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4)
for j in range(15):
    train_accuracy=0
    model.train()
    for train,labels in train_loader:
        out=model(train)
        pred=torch.argmax(out,dim=1)
        loss = loss_fn(out,labels)
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()
        train_accuracy+=(pred== labels).sum()
    print(" train set  accuracy: {}".format(train_accuracy/len(train_indices)))
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for test ,labels  in validation_loader: 
            outputs = model(test)
            pred=torch.argmax(outputs,dim=1)
            loss = loss_fn(outputs,labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (pred == labels).sum()
            total_accuracy = total_accuracy + accuracy
         


 train set  accuracy: 0.3125
 train set  accuracy: 0.3541666567325592
 train set  accuracy: 0.3854166567325592
 train set  accuracy: 0.4270833432674408
 train set  accuracy: 0.4479166567325592
 train set  accuracy: 0.4479166567325592
 train set  accuracy: 0.4375
 train set  accuracy: 0.4375
 train set  accuracy: 0.4375
 train set  accuracy: 0.46875
 train set  accuracy: 0.4791666567325592
 train set  accuracy: 0.5104166865348816
 train set  accuracy: 0.5208333134651184
 train set  accuracy: 0.5
 train set  accuracy: 0.5


In [154]:
#cnn
model=Simple_CNN(1,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4)
for j in range(15):
    train_accuracy=0
    model.train()
    for train,labels in train_loader:
        out=model(train)
        pred=torch.argmax(out,dim=1)
        loss = loss_fn(out,labels)
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()
        train_accuracy+=(pred== labels).sum()
    print(" train set  accuracy: {}".format(train_accuracy/len(train_indices)))
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for test ,labels  in validation_loader: 
            outputs = model(test)
            pred=torch.argmax(outputs,dim=1)
            loss = loss_fn(outputs,labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (pred == labels).sum()
            total_accuracy = total_accuracy + accuracy

 train set  accuracy: 0.25
 train set  accuracy: 0.3125
 train set  accuracy: 0.2916666567325592
 train set  accuracy: 0.28125
 train set  accuracy: 0.3020833432674408
 train set  accuracy: 0.3020833432674408
 train set  accuracy: 0.3125
 train set  accuracy: 0.3333333432674408
 train set  accuracy: 0.2916666567325592
 train set  accuracy: 0.34375
 train set  accuracy: 0.3541666567325592
 train set  accuracy: 0.3125
 train set  accuracy: 0.3229166567325592
 train set  accuracy: 0.34375
 train set  accuracy: 0.3333333432674408


In [155]:
#多尺度
model=MDB_class(1,10,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4)
for j in range(15):
    train_accuracy=0
    model.train()
    for train,labels in train_loader:
        out=model(train,10)
        pred=torch.argmax(out,dim=1)
        loss = loss_fn(out,labels)
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()
        train_accuracy+=(pred== labels).sum()
    print(" train set  accuracy: {}".format(train_accuracy/len(train_indices)))
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for test ,labels  in validation_loader: 
            outputs = model(test,10)
            pred=torch.argmax(outputs,dim=1)
            loss = loss_fn(outputs,labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (pred == labels).sum()
            total_accuracy = total_accuracy + accuracy

 train set  accuracy: 0.25
 train set  accuracy: 0.4583333432674408
 train set  accuracy: 0.4895833432674408
 train set  accuracy: 0.6041666865348816
 train set  accuracy: 0.6979166865348816
 train set  accuracy: 0.6979166865348816
 train set  accuracy: 0.71875
 train set  accuracy: 0.7708333134651184
 train set  accuracy: 0.78125
 train set  accuracy: 0.7916666865348816
 train set  accuracy: 0.7916666865348816
 train set  accuracy: 0.78125
 train set  accuracy: 0.7916666865348816
 train set  accuracy: 0.7916666865348816
 train set  accuracy: 0.7916666865348816
